In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
# Load data 
wd = "/Users/antonaleynikov/Desktop/Fixed Income Sentiment strat"
os.chdir(wd)
os.getcwd()

'/Users/antonaleynikov/Desktop/Fixed Income Sentiment strat'

In [4]:
for path, folders, files in os.walk(wd):
    print("folders:", folders)
    print("files:", files)
    print("path:", path)

folders: ['Papers', '.venv', 'Data', 'Bond indices']
files: ['FINANCE Fixed-Income Securities 0470852771.pdf', '.Rhistory', '.DS_Store', 'US_Bonds_Butterflies_Strat.ipynb', 'butterfly (understanding).pdf', 'stochvol_script.Rmd', 'stochvol_script.nb.html', 'Bond butterfly model testing.Rmd', 'quantnotebook.Rmd']
path: /Users/antonaleynikov/Desktop/Fixed Income Sentiment strat
folders: []
files: ['Bonds butterfly strategy papers.zip', '.DS_Store', 'Nelson-Siegel.pdf', 'Guido.pdf', 'Fabozzi Butterflies.pdf', 'Diebold-Li.pdf', 'Yield Curve Trading Strategies Exploiting Sentiment Data.pdf']
path: /Users/antonaleynikov/Desktop/Fixed Income Sentiment strat/Papers
folders: ['bin', 'include', 'lib', 'share']
files: ['pyvenv.cfg']
path: /Users/antonaleynikov/Desktop/Fixed Income Sentiment strat/.venv
folders: []
files: ['pip3.9', 'jupyter-run', 'Activate.ps1', 'python3', 'python', 'pip3', 'ipython', 'activate.fish', 'wheel', 'python3.9', 'f2py', 'ipython3', 'pip', 'jupyter-troubleshoot', 'pygmen

In [22]:
data_path = os.path.join(wd, "Data")
datafiles_names = os.listdir(wd + "/Data")

filenames = [filename.split(".") for filename in datafiles_names[2:]]

In [24]:
for file in filenames:
    file_name = file[0]
    file_format = file[1]
    file_path = os.path.join(data_path , ".".join(file))
    df = pd.read_excel(file_path, index_col = "Dates")
    if file_format == "xlsx":
        df = pd.read_excel(file_path, index_col = "Dates")
    globals()[file_name] = df 
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import numpy as np\nimport pandas as pd\nimport os',
  'import numpy as np\nimport pandas as pd\nimport os',
  '# Load data \nwd = "/Users/antonaleynikov/Desktop/Fixed Income Sentiment strat"\nos.chdir(wd)\nos.getcwd()',
  'for path, folders, files in os.walk(wd):\n    print("folders:", folders)\n    print("files:", files)\n    print("path:", path)',
  'data_path = os.path.join(wd, "Data")\ndatafiles_names = os.listdir(wd + "/Data")\ndatafiles_names = datafiles_names[1:]\n\nfilenames = [filename.split(".") for filename in datafiles_names]',
  'for file in filenames:\n    file_name = file[0]\n    file_format = file[1]\n    file_path = os.path.join(data_path , ".".join(file))\n    if file_format == "xlsx":\n     

In [25]:
Yields.head()

,DGS1MO,DGS3MO,DGS6MO,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS20,DGS30
Dates,,,,,,,,,,,
2003-01-02,1.18,1.22,1.25,1.42,1.80,2.22,3.05,3.62,4.07,5.05,5.18
2003-02-03,1.16,1.18,1.21,1.34,1.72,2.17,3.05,3.58,4.01,4.93,5.01
2003-03-03,1.20,1.20,1.21,1.26,1.52,1.89,2.66,3.21,3.68,4.69,4.92
2003-04-01,1.17,1.12,1.10,1.16,1.50,1.92,2.78,3.35,3.84,4.85,5.06
2003-05-01,1.09,1.10,1.13,1.21,1.50,1.93,2.82,3.37,3.88,4.80,5.01


In [86]:
Exogen["SPX_EP_YIELD_SPREAD"] = Exogen["SPX_PE_RATIO"].apply(lambda x: 1 / x) - (Yields[1:]["DGS20"]/100).values
del Exogen["SPX_PE_RATIO"]
Exogen

,unk,USYC3M10 Index,SPX_GROSS_AGGTE_DVD_YLD,SPX_EP_YIELD_SPREAD
Dates,,,,
2003-02-28,-0.45,249.704,1.90,0.001084
2003-03-31,-0.18,269.405,1.88,0.019077
2003-04-30,-0.65,272.061,1.75,0.011914
2003-05-31,0.01,226.873,1.67,0.005934
2003-06-30,0.12,266.133,1.66,0.008927
...,...,...,...,...
2022-11-30,-0.55,-77.371,1.64,0.008100
2022-12-31,-0.47,-53.018,1.76,0.016390
2023-01-31,0.41,-118.137,1.66,0.011095


In [43]:
from sklearn.linear_model import LinearRegression as lm 


def get_lam(yields, lam_seq):
    "get lambda given known yields and a searching grid"
    if 0 in lam_seq:
        print("Remove zero from search grid")
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    r2 = pd.DataFrame(0, index=np.arange(yields.shape[0]), columns=lam_seq)
    for j in range(len(lam_seq)):
        lam = lam_seq[j]
        
        # Initialise regressors based on lambda 
        regs = pd.DataFrame()
        regs["level_reg"] = [1 for t in maturities]
        regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
        regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
        
        # Fit yields to regressors
        factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
        for i in range(yields.shape[0]):
            X = regs.values
            y = yields.iloc[i].values
            reg = lm().fit(X, y)
            level, slope ,curv = reg.intercept_, reg.coef_[1], reg.coef_[2]
            rsq = reg.score(X,y)
            factors.iloc[i] = [level, slope, curv]
            r2.iloc[i,j] = rsq
    lam_max = r2.mean().idxmax()
    return lam_max
        
# get_lam(yields, np.linspace(0.1,1,10))

def fit_factors(yields, lam):
    factors = pd.DataFrame({"level": "", "slope": "", "curvature": ""},index = yields.index)
    
    # Initialise regressors based on lambda
    maturities = np.array([1/12, 1/4,1/2 ,1 , 2, 3, 5, 7, 10, 20, 30])
    regs = pd.DataFrame()
    regs["level_reg"] = [1 for t in maturities]
    regs["slope_reg"] = [(1 - np.exp(-lam * t))/(lam * t) for t in maturities]
    regs["curv_reg"] = [(1 - np.exp(-lam * t))/(lam * t) - np.exp(-lam*t) for t in maturities]
    
    for row in range(factors.shape[0]):
        reg = lm().fit(regs.values, yields.iloc[row].values)
        level = reg.intercept_
        slope , curv = reg.coef_[1], reg.coef_[2]
        factors.iloc[row] = [level, slope, curv]
    return factors

# lam = get_lam(yields, np.linspace(0.1,1,10))
# factors = fit_factors(yields, lam)

In [55]:
lam = get_lam(Yields, np.linspace(0.1,1,10))
factors = fit_factors(Yields, lam)
dfact = factors.diff()[1:]

In [87]:
# TEST
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
from rpy2.robjects import pandas2ri
utils = importr("utils")
# utils.chooseCRANmirror(ind=1)
# utils.install_packages('rugarch')
# utils.install_packages('forecast')
forecast = importr('forecast')
rugarch = importr('rugarch')
base = importr('base')

# X is a Panda Series that contains the returns I want to fit the model
returns = FloatVector(dfact["slope"].values)   
with (ro.default_converter + pandas2ri.converter).context():
     exo = ro.conversion.get_conversion().py2rpy(Exogen)

ro.r("""  
     f <- function (series, exogen) {
     spec = ugarchspec(
     variance.model=list(garchOrder=c(1,1)),
     mean.model=list(armaOrder=auto.arima(series)$arma[1:2], external.regressors = exogen)
     ) 

     fit = tryCatch(
     ugarchfit(
     spec, series, solver = 'hybrid'
     ), error=function(e) e, warning=function(w) w
     )  
     fore = ugarchforecast(fit, n.ahead=1)
     ind = fore@forecast$seriesFor
     ind
     }""")

fcast_armax = ro.globalenv['f']
fore = fcast_armax(returns, base.as_matrix(exo))[0]

fore

0.1416401780725572

In [91]:
s_len = dfact.shape[0] 
win = 160
h = 1
arr = []
for t in range (s_len - win - h):
    end = t + win

    series = FloatVector(dfact["slope"][t:end].values)   
    with (ro.default_converter + pandas2ri.converter).context():
        exo = ro.conversion.get_conversion().py2rpy(Exogen[t:end])
    
    fore = fcast_armax(series, base.as_matrix(exo))[0]
    arr.append(fore)

[0.19317608539552092,
 0.18018926916865224,
 0.17255158614264943,
 0.18232294526852627,
 0.17871204617450254,
 0.4279646544242296,
 0.1520407653447674,
 0.2628515975980475,
 0.2609184146289019,
 0.24469935330648174,
 0.25254443855236025,
 0.2386124370540521,
 0.2520611237113613,
 0.2378937101087396,
 0.23553212401674475,
 0.2411587648791841,
 0.2675869934060804,
 0.3175757939095609,
 0.329173491353995,
 0.22235645101003432,
 0.33462076955880443,
 0.21485972025369443,
 0.20802424595132274,
 0.27081360981480784,
 0.19358132396396013,
 0.1995986401568055,
 0.19833761287934956,
 0.19820930620474805,
 0.20837887948619754,
 0.22596191577449923,
 0.09338435339709697,
 0.15663770560994805,
 0.19537391438517085,
 0.16786757591761847,
 0.06570746711764393,
 0.0970488750228106,
 0.11554247762608058,
 0.04134560560079363,
 0.1678397708568549,
 0.12529960945341775,
 0.2152224588995133,
 0.039926793698944744,
 0.16074503780879684,
 0.15740605349770567,
 0.09126436213765626,
 0.1664857845614406,
 0.1